In [4]:
import praw

import csv
from datetime import datetime
import pandas as pd
import datetime as dt
import pickle

import re
import string

import nltk
from nltk import word_tokenize, pos_tag


In [2]:
reddit = praw.Reddit(client_id='0yClWPotAY79LjR16mcelQ',
                     client_secret='pTZzcYC9LfrI5YLe61qw6-5KaZLdtQ',
                     password='blahblah@123',
                     user_agent='Get Reddit data 1.0 by /u/coffeeandcode1111',
                     username='coffeeandcode1111')

In [ ]:
def setup(id, secret, password, user_agent, username):
    reddit = praw.Reddit(client_id = id,
                     client_secret = secret,
                     password = password,
                     user_agent = user_agent,
                     username = username)

In [3]:
def get_reddit_data(subreddit_name, max_count):
    """Scrapes Reddit submissions and comments.
    
    Parameters
    ----------
    subreddit_name : string
        The subreddit name.
    max_count : int
        The maximum number of posts to query.
    """
    filename = subreddit_name + '_' + str(max_count) + '_' + datetime.now().strftime('%Y%m%d') + '.csv'
    # Setting up a csv writer and write the first row 
    writer = csv.writer(open(filename, 'wt', encoding = 'utf-8'))
    writer.writerow(['idstr', 'created', 'created_datetime', 'nsfw', 'flair_text', 'flair_css_class',
                     'author', 'title', 'selftext', 'score', 'upvote_ratio', 
                     'distinguished', 'textlen', 'num_comments', 'top_comments'])   
    item_count = 0
    comment_count = 0
    for submission in reddit.subreddit(subreddit_name).hot(limit=None): 
        try:
            item_count += 1
            idstr = submission.id
            created = submission.created
            created_datetime = datetime.fromtimestamp(created).strftime('%Y' + '-' + '%m' + '-' + '%d')
            nsfw = submission.over_18
            flair_text = submission.link_flair_text
            flair_css_class = submission.link_flair_css_class
            author = submission.author
            title = submission.title
            selftext = submission.selftext
            score = submission.score
            upvote_ratio = submission.upvote_ratio
            distinguished = submission.distinguished
            textlen = len(submission.selftext)
            num_comments = submission.num_comments
            comment_list = []
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                if comment.author != None:
                    comment_count += 1
                    comment_list.append(comment.body)
            comments = ' '.join(comment_list)
            writer.writerow((idstr, created, created_datetime, nsfw, flair_text, flair_css_class,
                             author, title, selftext, score, upvote_ratio,
                             distinguished, textlen, num_comments, comments))
            print('.', end='', flush=True)
        except:
            print('Error found--resuming...')
        if item_count == max_count:
            break

    if item_count > 0:
        print('Done!' + '\n' + 'Found ' + str(item_count) + ' posts' + 
              '\n' + 'Found ' + str(comment_count) + ' comments')


In [4]:
get_reddit_data('Futurology', 1000)

.....................................................................................................................................................................................................................................................................................................Error found--resuming...
.........

In [7]:
def get_reddit_data_1(subreddit_name, max_count):
    """Scrapes Reddit submissions and comments.
    
    Parameters
    ----------
    subreddit_name : string
        The subreddit name.
    max_count : int
        The maximum number of posts to query.
    """
    filename = 'controversial_'+ subreddit_name + '_' + str(max_count) + '_' + datetime.now().strftime('%Y%m%d') + '.csv'
    # Setting up a csv writer and write the first row 
    writer = csv.writer(open(filename, 'wt', encoding = 'utf-8'))
    writer.writerow(['idstr', 'created_datetime', 'flair_text', 'flair_css_class',
                     'author', 'title', 'selftext', 
                     'distinguished', 'textlen'])   
    item_count = 0
    comment_count = 0
    for submission in reddit.subreddit(subreddit_name).controversial(limit=None): 
        try:
            item_count += 1
            idstr = submission.id
            created = submission.created
            created_datetime = datetime.fromtimestamp(created).strftime('%Y' + '-' + '%m' + '-' + '%d')
            flair_text = submission.link_flair_text
            flair_css_class = submission.link_flair_css_class
            author = submission.author
            title = submission.title
            selftext = submission.selftext
            distinguished = submission.distinguished
            textlen = len(submission.selftext)
            writer.writerow((idstr, created_datetime, flair_text, flair_css_class,
                             author, title, selftext,
                             distinguished, textlen))
            print('.', end='', flush=True)
        except:
            print('Error found--resuming...')
        if item_count == max_count:
            break

In [8]:
get_reddit_data_1('Futurology', 2000)

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

DATA PREPROCESSING

In [ ]:
#NOW - LATEST
for key in submissions_info.keys():
    filename = '1_submission_id_'+ key + "_submissions.csv"
    with open(filename, 'r') as f:
        df = pd.read_csv(filename)
        rem_filename = '1_rem_submission_id_'+ key + "_submissions.csv"
        mod_filename = '1_mod_submission_id_'+ key + "_submissions.csv"
        removed_df = df[(df['selftext'] == "[removed]") | (df['selftext'] == "[deleted]")]
        removed_df.to_csv(rem_filename, index=False)
        cond1 = df['selftext'].isin(removed_df['selftext'])
        print("Month", key, "Size before exclusion drop", df.shape)
        df.drop(df[cond1].index, inplace = True)
        print("Month", key, "Size before len check", df.shape)
        cond2 = (df['selftext'].str.len() < 30)
        df.drop(df[cond2].index, inplace = True)
        print("Month", key, "Size after len check", df.shape)
        df.to_csv(mod_filename, index=False)

DATA CLEANING

In [1]:
def pos_filter(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives'''
    is_noun_or_adj = lambda pos: pos[:2] in ['NN', 'JJ', 'CD', 'NNP', 'RB', 'VB']
    tokenized = word_tokenize(text)
    all_nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_or_adj(pos)] 
    return ' '.join(all_nouns_adj)

In [2]:
def clean_text(text):
    my_punctuation = string.punctuation
    my_punctuation += '’' #frequent occurence in data
    my_stopwords = nltk.corpus.stopwords.words('english')
    
    text_list = []
    
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('…', '', text)
    text = re.sub('['+my_punctuation +']+', ' ', text)

    for word in text.split(' '):
        if word in my_stopwords or len(word) == 1:
            word = ""
            continue
        text_list.append(word)
    text = ' '.join(text_list)
    return text

In [3]:
import numpy as np

In [5]:
filename = '1ControversialPostsFuturology.csv'
with open(filename, 'r') as f:
    df = pd.read_csv(filename)
    df['selftext'] = np.where(df['selftext'].isna(), df['title'], df['selftext'])

In [19]:
df.to_csv('1ControversialPostsFuturology.csv')

In [6]:
df['selftext'] = df['selftext'].apply(pos_filter)
df['selftext'] = df['selftext'].apply(clean_text)

In [7]:
df.to_csv('CleanedControversialPostsFuturology.csv')

In [1]:
df.head()

NameError: name 'df' is not defined

TOPIC MODELLING

In [ ]:
comments = df.top_comments.to_list()
timestamps = df['created_utc'].apply(lambda x: pd.Timestamp(x))
topic_model = BERTopic(n_gram_range=(1,2))
topics, _ = topic_model.fit_transform(comments)
topics_over_time = topic_model.topics_over_time(docs=comments, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topics()

In [ ]:
topic_model.visualize_topics_over_time()

In [ ]:
topic_model.visualize_barchart()

LABELLING EACH TOPIC GROUP

In [ ]:
dec_df['Topics'] = dec_df['selftext'].apply(lambda x: dec_topic_model.find_topics(x)[0][0])

#save to csv
dec_df.to_csv('TM_2021-12-24_submissions.csv')